In [ ]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz
%pip install spacy
%pip install sklearn
%pip install zemberek-python
%pip install trnltk
%pip install git+https://github.com/emres/turkish-deasciifier.git

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated

ERROR: Could not find a version that satisfies the requirement poyraz (from versions: none)
ERROR: No matching distribution found for poyraz


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
#!/usr/bin/env python
#-*-coding:utf-8-*-
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
import logging
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
from trnlp import TrnlpWord
from turkish.deasciifier import Deasciifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]   
            #DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]            
            #DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
                
            #DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan, result_type="new", lang='tr')
            results=[]
            for item in cursor.items(10):
                results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            getTweets.TweetClear(DataSet)
            getTweets.SentenceNormalization(DataSet)
            getTweets.TweetTranslate(DataSet)
            getTweets.KelimeAyristirma(DataSet) 
            DataSet.to_csv('tw.csv',index=False)
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            getTweets.TextblobSentiment(DataSet)

    def TextblobSentiment(DataSet):
        try:
            print ("Duygu Analizi Yapılıyor...")
            Veri=DataSet
            Veri["Tür"] = Veri["Tweet_Ayrı"].apply(getTweets.getPolarity)
            Veri["Yoğunluk"] = Veri["Tweet_Ayrı"].apply(getTweets.getSubjectivity)
            Veri.to_csv('textblob_eng_duygu.csv',index=False)
            print("Duygu Analizi Yapıldı!")                        
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    """def TrSentiment(DataSet):
        try:
            print("Türkçe Duygu Analizi Yapılıyor...")
            for example,index in DataSet["Tweet_Degistirilmis"]:
                DataSet["TR_A"][index] = simple_token(example, sw=stopwords)
            print("Türkçe Duygu Analizi Yapıldı!")
        except Exception as err:
            print('Error: {}'.format(err)) """
                     
    def TweetTranslate(DataSet):
        try:
            print ("Tweetler Çeviriliyor...")
            for yazi in range(len(DataSet["Tweet_Degistirilmis"])):
                Ceviri_Blob=TextBlob(DataSet["Tweet_Degistirilmis"][yazi])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                DataSet["Tweet_Ceviri"][yazi]=Eng_Blob
                DataSet["Tweet_Ceviri"][yazi]=DataSet["Tweet_Ceviri"][yazi].lower()
            print ("Tweetler Çevirildi!")                      
            return DataSet        
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TweetClear(DataSet):
        try:
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n+|RT+|&amp')
            for i in range(len(DataSet["Tweet_Degistirilmis"])):        
                DataSet["Tweet_Degistirilmis"][i]=pattern.sub('', DataSet["Tweet_Degistirilmis"][i])    
                DataSet["Tweet_Degistirilmis"][i]=DataSet["Tweet_Degistirilmis"][i].lower()              
            DataSet["Tweet_Degistirilmis"]=DataSet["Tweet_Degistirilmis"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
            
    def SentenceNormalization(DataSet):
        try:
            morphology = TurkishMorphology.create_with_defaults()
            normalizer = TurkishSentenceNormalizer(morphology)

            start = time.time()
            for example,index in DataSet["Tweet_Degistirilmis"]:
                 DataSet["Tweet_Degistirilmis"][index]= normalizer.normalize(example)
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
            
    def KelimeAyristirma(DataSet):
        try:
            stop = stopwords.words('english')
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ceviri"].str.split()
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ayrı"].apply(lambda x: [item for item in x if item not in stop])
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ayrı"].str.join(" ")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
        
    def getSubjectivity(DataSet):
        try: 
            sub=TextBlob(DataSet).sentiment.subjectivity               
            if sub >=0.75:
                return "Kesinlikle"
            elif sub >=0.5 and sub <0.75:
                return "Yüksek İhtimal"
            elif sub >=0.25 and sub <0.50:
                return "Düşük İhtimal"
            else:
                return "Kesin Değil"
        except Exception as err:
            print('Error: {}'.format(err))
            
    def getPolarity(DataSet):
        try:
            pol=TextBlob(DataSet).sentiment.polarity
            if pol>0.1:
                return "Pozitif"
            elif pol<-0.1:
                return "Negatif"
            else:
                return "Nötr"
        except Exception as err:
            print('Error: {}'.format(err))

In [11]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
2023-01-10 03:16:22,120 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 8.251937627792358

Error: too many values to unpack (expected 2)
Tweetler Çeviriliyor...
Tweetler Çevirildi!
Duygu Analizi Yapılıyor...
Duygu Analizi Yapıldı!


In [12]:
Veri=pd.read_csv('textblob_eng_duygu.csv')
Veri.head()

,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı,Tür,Yoğunluk
0,Janel 📖,2023-01-10 00:16:10+00:00,269343888 defneyaprağıacaktım etyaranacaklar kara sabanacaklar glütenecektim gücendiriciecektim formülleştirmekebil… https://t.co/sJmsMsJtED,defneyaprağıacaktım etyaranacaklar kara sabanacaklar glütenecektim gücendiriciecektim formülleştirmekebil,i was going to laurel.,going laurel.,Nötr,Kesin Değil
1,MEHMET AKIF DEMIR,2023-01-10 00:15:22+00:00,@bbcturkce Ozon delindi diyorlar tamam! Ozon yama tuttu diyorlar tamam!Kara delik keşfettik diyorlar tamam! Uzayın… https://t.co/LOJh18iSsY,ozon delindi diyorlar tamam ozon yama tuttu diyorlar tamamkara delik keşfettik diyorlar tamam uzayın,"they say ozone is punctured, they say ozone kept patch.","say ozone punctured, say ozone kept patch.",Nötr,Kesin Değil
2,Hakan Kanat خاقان قناد,2023-01-10 00:14:29+00:00,"RT @baarrss__: Bu işler böyledir; Halep demezseniz Hatay’ı tartışmaya açarlar, Musul-Kerkük demezseniz Diyarbakır’a göz koyarlar, Selanik d…",bu işler böyledir halep demezseniz hatayı tartışmaya açarlar musulkerkük demezseniz diyarbakıra göz koyarlar selanik d,"if you do not say aleppo, they will open the mistake if you don't say mosulkerkuk, they put an eye on diyarbakır.","say aleppo, open mistake say mosulkerkuk, put eye diyarbakır.",Nötr,Yüksek İhtimal
3,ŞAH-I MERDAN,2023-01-10 00:14:24+00:00,"RT @KarasoyMurad: Kıymetli Dostlar,\nyurtdışına çıkmak isterken açılan ateş sonrası vefat eden KHK’lının, faşist iktidarın kara propaganda a…",kıymetli dostlaryurtdışına çıkmak isterken açılan ateş sonrası vefat eden khklının faşist iktidarın kara propaganda a,dear friends who died after the fire opened while trying to go to the fascist power of the black propaganda a,dear friends died fire opened trying go fascist power black propaganda,Negatif,Düşük İhtimal
4,...,2023-01-10 00:13:56+00:00,"Seni çok seviyorum n'e'fesim. 🖤\nBiliyorum hep benimlesin..\nGülüşün, bakışların, kokun, nefesin..\nBu acı hiç geçmedi… https://t.co/pIR2WJVxb5",seni çok seviyorum nefesim biliyorum hep benimlesingülüşün bakışların kokun nefesinbu acı hiç geçmedi,"i love you so much, i know my breath, your gaze is always your breath, your breath has never passed","love much, know breath, gaze always breath, breath never passed",Pozitif,Düşük İhtimal


In [13]:
Veri1=pd.read_csv('tw.csv')
Veri1.head()

,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı
0,Janel 📖,2023-01-10 00:16:10+00:00,269343888 defneyaprağıacaktım etyaranacaklar kara sabanacaklar glütenecektim gücendiriciecektim formülleştirmekebil… https://t.co/sJmsMsJtED,defneyaprağıacaktım etyaranacaklar kara sabanacaklar glütenecektim gücendiriciecektim formülleştirmekebil,i was going to laurel.,going laurel.
1,MEHMET AKIF DEMIR,2023-01-10 00:15:22+00:00,@bbcturkce Ozon delindi diyorlar tamam! Ozon yama tuttu diyorlar tamam!Kara delik keşfettik diyorlar tamam! Uzayın… https://t.co/LOJh18iSsY,ozon delindi diyorlar tamam ozon yama tuttu diyorlar tamamkara delik keşfettik diyorlar tamam uzayın,"they say ozone is punctured, they say ozone kept patch.","say ozone punctured, say ozone kept patch."
2,Hakan Kanat خاقان قناد,2023-01-10 00:14:29+00:00,"RT @baarrss__: Bu işler böyledir; Halep demezseniz Hatay’ı tartışmaya açarlar, Musul-Kerkük demezseniz Diyarbakır’a göz koyarlar, Selanik d…",bu işler böyledir halep demezseniz hatayı tartışmaya açarlar musulkerkük demezseniz diyarbakıra göz koyarlar selanik d,"if you do not say aleppo, they will open the mistake if you don't say mosulkerkuk, they put an eye on diyarbakır.","say aleppo, open mistake say mosulkerkuk, put eye diyarbakır."
3,ŞAH-I MERDAN,2023-01-10 00:14:24+00:00,"RT @KarasoyMurad: Kıymetli Dostlar,\nyurtdışına çıkmak isterken açılan ateş sonrası vefat eden KHK’lının, faşist iktidarın kara propaganda a…",kıymetli dostlaryurtdışına çıkmak isterken açılan ateş sonrası vefat eden khklının faşist iktidarın kara propaganda a,dear friends who died after the fire opened while trying to go to the fascist power of the black propaganda a,dear friends died fire opened trying go fascist power black propaganda
4,...,2023-01-10 00:13:56+00:00,"Seni çok seviyorum n'e'fesim. 🖤\nBiliyorum hep benimlesin..\nGülüşün, bakışların, kokun, nefesin..\nBu acı hiç geçmedi… https://t.co/pIR2WJVxb5",seni çok seviyorum nefesim biliyorum hep benimlesingülüşün bakışların kokun nefesinbu acı hiç geçmedi,"i love you so much, i know my breath, your gaze is always your breath, your breath has never passed","love much, know breath, gaze always breath, breath never passed"
